In [4]:

import os
os.environ['nvcc_path'] = 'E:/Cuda/Cuda Dev/bin/nvcc'

import jittor as jt
from jittor import init
import argparse
import numpy as np
import math
from jittor import nn




In [5]:
if jt.has_cuda:
    jt.flags.use_cuda = 1
    print("Using CUDA.")
    

Using CUDA.


In [6]:
python -m jittor.test.test_example

SyntaxError: invalid syntax (2912452638.py, line 1)